# Celebrity Data

In [6]:
#Importing dependencies
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
from sklearn.model_selection import train_test_split

In [7]:
#Adding allowed extention for photos
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [13]:
#Setting prediction model

def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # Load image file and find face locations
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)    
    are_matches = []
    for i in range(len(X_face_locations)):
        print(closest_distances[0][i][0])
        are_matches.append(closest_distances[0][i][0] <= distance_threshold)
        
    # Predict classes and remove classifications that aren't within the threshold
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


In [16]:
#Showing the face recognition results on the image
def show_prediction_labels_on_image(img_path, predictions):
  
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # There's a bug in Pillow where it blows up with non-UTF-8 text
        # when using the default bitmap font
        name = name.encode("UTF-8")

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
    pil_image.show()

In [17]:
if __name__ == "__main__":
    # Using the trained classifier, make predictions for unknown images
    for image_file in os.listdir("./faces/test"):
        full_file_path = os.path.join("./faces/test", image_file)

        print("Looking for faces in {}".format(image_file))

        # Find all people in the image using a trained classifier model
        # Note: You can pass in either a classifier file name or a classifier model instance
        predictions = predict(full_file_path, model_path="trained_knn_model.clf")

        # Print results on the console
        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))

        # Display results overlaid on an image
        show_prediction_labels_on_image(os.path.join("./faces/test", image_file), predictions)


Looking for faces in 7cwSSZJA.jpg
0.4980958050192508
- Found 1878 at (142, 43)
Looking for faces in 1366_v9_bb.jpg
0.5308956890980153
- Found 3383 at (280, 428)
Looking for faces in tmp6boni0ss copy.png
0.47844539674331427
- Found 8240 at (741, 1380)
Looking for faces in aaron-paul-435-63.jpg
0.29975917611213315
- Found 18 at (153, 110)
Looking for faces in image4.jpg
0.4866149909385523
- Found 4136 at (379, 171)
Looking for faces in IMG_20190728_170400.jpg
Looking for faces in image1.jpg
0.5294938784818516
- Found 555 at (161, 162)
Looking for faces in IMG_20190728_170410.jpg
Looking for faces in tmp6i_64do3 copy.png
0.5079050665259941
- Found 6509 at (1124, 1125)
